In [59]:
import dask.dataframe as ddf
import pandas as pd
import datetime
import functools
import seaborn as sns
import matplotlib.pyplot as plt

In [60]:
year = 1994

In [61]:
standard_pop = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/Lung_cancer/2000_standard_population.csv')
standard_pop

,Age,Number,Weight
0,All ages,1000000,1.000000
1,Under 1 year,13818,0.013818
2,1-4 years,55317,0.055317
3,5-14 years,145565,0.145565
4,15-24 years,138646,0.138646
5,25-34 years,135573,0.135573
6,35-44 years,162613,0.162613
7,45-54 years,134834,0.134834
8,55-64 years,87247,0.087247
9,65-74 years,66037,0.066037


In [62]:
pop = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/temp_bins_suicide/Population/population_detailed_1969-2020.csv', dtype = {'year': int, 'fips': str})
pop = pop.loc[:, ~pop.columns.str.contains('^Unnamed')]
pop

,year,state,fips,race,sex,age,pop
0,1969,AL,01001,white,Male,0 years,159
1,1969,AL,01001,white,Male,1-4 years,657
2,1969,AL,01001,white,Male,5-9 years,1137
3,1969,AL,01001,white,Male,10-14 years,956
4,1969,AL,01001,white,Male,15-19 years,721
...,...,...,...,...,...,...,...
15536023,2020,WY,56045,other,Female,65-69 years,21
15536024,2020,WY,56045,other,Female,70-74 years,2
15536025,2020,WY,56045,other,Female,75-79 years,2
15536026,2020,WY,56045,other,Female,80-84 years,2


In [63]:
pop['age'].unique()

array(['0 years', '1-4 years', '5-9 years', '10-14 years', '15-19 years',
       '20-24 years', '25-29 years', '30-34 years', '35-39 years',
       '40-44 years', '45-49 years', '50-54 years', '55-59 years',
       '60-64 years', '65-69 years', '70-74 years', '75-79 years',
       '80-84 years', 'over 85 years'], dtype=object)

In [64]:
age_group1 = ['0 years']
age_group2 = ['1-4 years']
age_group3 = ['5-9 years', '10-14 years']
age_group4 = ['15-19 years', '20-24 years']
age_group5 = ['25-29 years', '30-34 years']
age_group6 = ['35-39 years', '40-44 years']
age_group7 = ['45-49 years', '50-54 years']
age_group8 = ['55-59 years', '60-64 years']
age_group9 = ['65-69 years', '70-74 years']
age_group10 = ['75-79 years', '80-84 years']
age_group11 = ['over 85 years']

In [65]:
def defineRange(age):
    if age in age_group1:
        return 'Under 1 year'
    elif age in age_group2:
        return '1-4 years'
    elif age in age_group3:
        return '5-14 years'
    elif age in age_group4:
        return '15-24 years'
    elif age in age_group5:
        return '25-34 years'
    elif age in age_group6:
        return '35-44 years'
    elif age in age_group7:
        return '45-54 years'
    elif age in age_group8:
        return '55-64 years'
    elif age in age_group9:
        return '65-74 years'
    elif age in age_group10:
        return '75-84 years'
    else:
        return '85 years and over'

In [66]:
pop['age_range'] = pop['age'].apply(defineRange)
pop

,year,state,fips,race,sex,age,pop,age_range
0,1969,AL,01001,white,Male,0 years,159,Under 1 year
1,1969,AL,01001,white,Male,1-4 years,657,1-4 years
2,1969,AL,01001,white,Male,5-9 years,1137,5-14 years
3,1969,AL,01001,white,Male,10-14 years,956,5-14 years
4,1969,AL,01001,white,Male,15-19 years,721,15-24 years
...,...,...,...,...,...,...,...,...
15536023,2020,WY,56045,other,Female,65-69 years,21,65-74 years
15536024,2020,WY,56045,other,Female,70-74 years,2,65-74 years
15536025,2020,WY,56045,other,Female,75-79 years,2,75-84 years
15536026,2020,WY,56045,other,Female,80-84 years,2,75-84 years


In [67]:
cancer = pd.read_csv('/global/cfs/cdirs/m1532/Projects_MVP/geospatial/Lung_cancer/Lung_Cancer_Mortality/lung_cancer_mortality/lung_cancer_mortality_' + str(year) + '.csv', 
                              dtype = {'year': int, 'county_residence': str, 'county_death': str, 'county_residence': str, 'state_residence': str})
cancer= cancer.loc[:, ~cancer.columns.str.contains('^Unnamed')]

In [68]:
cancer

,year,county_residence,state_residence,state_death,county_death,month,sex,race,age,age_range,death_cause
0,1994,01017,01,01,01017,Jan,Male,White,75-84 years,65 and older,lung cancer
1,1994,01017,01,01,01017,Jan,Male,White,65-74 years,65 and older,lung cancer
2,1994,01073,01,01,01073,Jan,Male,White,55-64 years,age 25-64,lung cancer
3,1994,01073,01,01,01073,Jan,Female,White,65-74 years,65 and older,lung cancer
4,1994,01073,01,01,01073,Jan,Male,White,45-54 years,age 25-64,lung cancer
...,...,...,...,...,...,...,...,...,...,...,...
149566,1994,30013,30,56,56033,Dec,Male,White,75-84 years,65 and older,lung cancer
149567,1994,56037,56,56,56037,Dec,Female,White,35-44 years,age 25-64,lung cancer
149568,1994,56037,56,56,56037,Dec,Male,White,75-84 years,65 and older,lung cancer
149569,1994,56037,56,56,56037,Dec,Male,White,55-64 years,age 25-64,lung cancer


In [69]:
cancer['age'].unique()

array(['75-84 years', '65-74 years', '55-64 years', '45-54 years',
       '35-44 years', '85 years and over', '25-34 years', '1-4 years',
       '5-14 years', '15-24 years', nan], dtype=object)

In [70]:
cancer = cancer.rename(columns = {'state_death': 'statefips', 'county_death': 'fips'})
cancer_death = cancer.drop(['month', 'age_range', 'state_residence', 'county_residence'], axis = 1)
cancer_death = cancer_death.groupby(['year', 'fips', 'statefips', 'sex', 'race', 'age']).count()
cancer_death = cancer_death.reset_index()
cancer_death.head()

,year,fips,statefips,sex,race,age,death_cause
0,1994,01001,01,Female,White,45-54 years,2
1,1994,01001,01,Female,White,65-74 years,2
2,1994,01001,01,Female,White,75-84 years,4
3,1994,01001,01,Male,Black,45-54 years,1
4,1994,01001,01,Male,White,55-64 years,4


In [71]:
cancer_death = cancer_death.rename(columns = {'death_cause': 'deaths'})
cancer_death

,year,fips,statefips,sex,race,age,deaths
0,1994,01001,01,Female,White,45-54 years,2
1,1994,01001,01,Female,White,65-74 years,2
2,1994,01001,01,Female,White,75-84 years,4
3,1994,01001,01,Male,Black,45-54 years,1
4,1994,01001,01,Male,White,55-64 years,4
...,...,...,...,...,...,...,...
24098,1994,56043,56,Male,White,45-54 years,1
24099,1994,56043,56,Male,White,55-64 years,3
24100,1994,56043,56,Male,White,65-74 years,1
24101,1994,56043,56,Male,White,75-84 years,1


In [72]:
cancer_death['age'].unique()

array(['45-54 years', '65-74 years', '75-84 years', '55-64 years',
       '35-44 years', '85 years and over', '25-34 years', '1-4 years',
       '5-14 years', '15-24 years'], dtype=object)

In [73]:
pop_year = pop.drop('age', axis = 1)
pop_year = pop_year.rename(columns = {'age_range': 'age'})
pop_year

,year,state,fips,race,sex,pop,age
0,1969,AL,01001,white,Male,159,Under 1 year
1,1969,AL,01001,white,Male,657,1-4 years
2,1969,AL,01001,white,Male,1137,5-14 years
3,1969,AL,01001,white,Male,956,5-14 years
4,1969,AL,01001,white,Male,721,15-24 years
...,...,...,...,...,...,...,...
15536023,2020,WY,56045,other,Female,21,65-74 years
15536024,2020,WY,56045,other,Female,2,65-74 years
15536025,2020,WY,56045,other,Female,2,75-84 years
15536026,2020,WY,56045,other,Female,2,75-84 years


### Calcuate age_adjust rates for all races and genders

In [74]:
cancer_death.head()

,year,fips,statefips,sex,race,age,deaths
0,1994,01001,01,Female,White,45-54 years,2
1,1994,01001,01,Female,White,65-74 years,2
2,1994,01001,01,Female,White,75-84 years,4
3,1994,01001,01,Male,Black,45-54 years,1
4,1994,01001,01,Male,White,55-64 years,4


In [75]:
all_death = cancer_death.drop(['sex', 'race'], axis = 1)
all_death = all_death.groupby(['year', 'statefips', 'fips', 'age']).sum()
all_death = all_death.reset_index()
all_death

,year,statefips,fips,age,deaths
0,1994,48,48001,35-44 years,1
1,1994,48,48001,45-54 years,3
2,1994,48,48001,55-64 years,7
3,1994,48,48001,65-74 years,6
4,1994,48,48001,75-84 years,6
...,...,...,...,...,...
12515,1994,48,48501,65-74 years,1
12516,1994,48,48501,85 years and over,1
12517,1994,48,48503,35-44 years,1
12518,1994,48,48503,75-84 years,1


In [76]:
pop_year.head()

,year,state,fips,race,sex,pop,age
0,1969,AL,01001,white,Male,159,Under 1 year
1,1969,AL,01001,white,Male,657,1-4 years
2,1969,AL,01001,white,Male,1137,5-14 years
3,1969,AL,01001,white,Male,956,5-14 years
4,1969,AL,01001,white,Male,721,15-24 years


In [77]:
all_pop = pop_year.drop(['sex', 'race'], axis = 1)
all_pop = all_pop.groupby(['year', 'fips', 'state', 'age']).sum()
all_pop = all_pop.reset_index()
all_pop = all_pop[all_pop['year'] == year]
all_pop

,year,fips,state,age,pop
853204,1994,01001,AL,1-4 years,2350
853205,1994,01001,AL,15-24 years,4747
853206,1994,01001,AL,25-34 years,5646
853207,1994,01001,AL,35-44 years,5065
853208,1994,01001,AL,45-54 years,4508
...,...,...,...,...,...
887676,1994,56045,WY,55-64 years,618
887677,1994,56045,WY,65-74 years,410
887678,1994,56045,WY,75-84 years,300
887679,1994,56045,WY,85 years and over,99


In [78]:
def findStateFips(fips):
    return fips[:2]

In [79]:
all_death_pop = all_death.merge(all_pop, on = ['year', 'fips', 'age'], how = 'right')
all_death_pop['deaths'] = all_death_pop['deaths'].fillna(0)
all_death_pop['statefips'] = all_death_pop['fips'].apply(findStateFips)
all_death_pop

,year,statefips,fips,age,deaths,state,pop
0,1994,01,01001,1-4 years,0.0,AL,2350
1,1994,01,01001,15-24 years,0.0,AL,4747
2,1994,01,01001,25-34 years,0.0,AL,5646
3,1994,01,01001,35-44 years,0.0,AL,5065
4,1994,01,01001,45-54 years,3.0,AL,4508
...,...,...,...,...,...,...,...
34856,1994,56,56045,55-64 years,0.0,WY,618
34857,1994,56,56045,65-74 years,2.0,WY,410
34858,1994,56,56045,75-84 years,0.0,WY,300
34859,1994,56,56045,85 years and over,0.0,WY,99


In [80]:
death_race = cancer_death.drop(['sex'], axis = 1)
death_race = death_race.groupby(['year', 'statefips', 'fips', 'age', 'race']).sum()
death_race = death_race.reset_index()
death_race

,year,statefips,fips,age,race,deaths
0,1994,48,48001,35-44 years,Black,1
1,1994,48,48001,45-54 years,Black,1
2,1994,48,48001,45-54 years,White,2
3,1994,48,48001,55-64 years,Black,3
4,1994,48,48001,55-64 years,White,4
...,...,...,...,...,...,...
16034,1994,48,48501,65-74 years,White,1
16035,1994,48,48501,85 years and over,White,1
16036,1994,48,48503,35-44 years,White,1
16037,1994,48,48503,75-84 years,White,1


In [81]:
death_race['race'].unique()

array(['Black', 'White', 'Other'], dtype=object)

In [82]:
race_pop = pop_year.drop(['sex'], axis = 1)
race_pop = race_pop.groupby(['year', 'fips', 'state', 'age', 'race']).sum()
race_pop = race_pop.reset_index()
race_pop = race_pop[race_pop['year'] == year]
race_pop

,year,fips,state,age,race,pop
2214817,1994,01001,AL,1-4 years,black,537
2214818,1994,01001,AL,1-4 years,other,14
2214819,1994,01001,AL,1-4 years,white,1799
2214820,1994,01001,AL,15-24 years,black,697
2214821,1994,01001,AL,15-24 years,other,31
...,...,...,...,...,...,...
2301462,1994,56045,WY,65-74 years,white,409
2301463,1994,56045,WY,75-84 years,white,300
2301464,1994,56045,WY,85 years and over,white,99
2301465,1994,56045,WY,Under 1 year,other,2


In [83]:
race_replace = {'black': 'Black', 'white': 'White', 'other': 'Other'}
race_pop = race_pop.replace(race_replace)
race_pop.head()

,year,fips,state,age,race,pop
2214817,1994,01001,AL,1-4 years,Black,537
2214818,1994,01001,AL,1-4 years,Other,14
2214819,1994,01001,AL,1-4 years,White,1799
2214820,1994,01001,AL,15-24 years,Black,697
2214821,1994,01001,AL,15-24 years,Other,31


In [84]:
race_death_pop = death_race.merge(race_pop, on = ['year', 'fips', 'age', 'race'], how = 'right')
race_death_pop['deaths'] = race_death_pop['deaths'].fillna(0)
race_death_pop['statefips'] = race_death_pop['fips'].apply(findStateFips)
race_death_pop

,year,statefips,fips,age,race,deaths,state,pop
0,1994,01,01001,1-4 years,Black,0.0,AL,537
1,1994,01,01001,1-4 years,Other,0.0,AL,14
2,1994,01,01001,1-4 years,White,0.0,AL,1799
3,1994,01,01001,15-24 years,Black,0.0,AL,697
4,1994,01,01001,15-24 years,Other,0.0,AL,31
...,...,...,...,...,...,...,...,...
87073,1994,56,56045,65-74 years,White,2.0,WY,409
87074,1994,56,56045,75-84 years,White,0.0,WY,300
87075,1994,56,56045,85 years and over,White,0.0,WY,99
87076,1994,56,56045,Under 1 year,Other,0.0,WY,2


In [85]:
death_gender = cancer_death.drop(['race'], axis = 1)
death_gender = death_gender.groupby(['year', 'statefips', 'fips', 'age', 'sex']).sum()
death_gender = death_gender.reset_index()
death_gender

,year,statefips,fips,age,sex,deaths
0,1994,48,48001,35-44 years,Male,1
1,1994,48,48001,45-54 years,Female,1
2,1994,48,48001,45-54 years,Male,2
3,1994,48,48001,55-64 years,Female,1
4,1994,48,48001,55-64 years,Male,6
...,...,...,...,...,...,...
19622,1994,48,48501,65-74 years,Female,1
19623,1994,48,48501,85 years and over,Male,1
19624,1994,48,48503,35-44 years,Female,1
19625,1994,48,48503,75-84 years,Male,1


In [86]:
gender_pop = pop_year.drop(['race'], axis = 1)
gender_pop = gender_pop.groupby(['year', 'fips', 'state', 'age', 'sex']).sum()
gender_pop = gender_pop.reset_index()
gender_pop = gender_pop[gender_pop['year'] == year]
gender_pop

,year,fips,state,age,sex,pop
1706214,1994,01001,AL,1-4 years,Female,1157
1706215,1994,01001,AL,1-4 years,Male,1193
1706216,1994,01001,AL,15-24 years,Female,2385
1706217,1994,01001,AL,15-24 years,Male,2362
1706218,1994,01001,AL,25-34 years,Female,2938
...,...,...,...,...,...,...
1775152,1994,56045,WY,75-84 years,Male,116
1775153,1994,56045,WY,85 years and over,Female,71
1775154,1994,56045,WY,85 years and over,Male,28
1775155,1994,56045,WY,Under 1 year,Female,36


In [87]:
gender_death_pop = death_gender.merge(gender_pop, on = ['year', 'fips', 'age', 'sex'], how = 'right')
gender_death_pop['deaths'] = gender_death_pop['deaths'].fillna(0)
gender_death_pop['statefips'] = gender_death_pop['fips'].apply(findStateFips)
gender_death_pop

,year,statefips,fips,age,sex,deaths,state,pop
0,1994,01,01001,1-4 years,Female,0.0,AL,1157
1,1994,01,01001,1-4 years,Male,0.0,AL,1193
2,1994,01,01001,15-24 years,Female,0.0,AL,2385
3,1994,01,01001,15-24 years,Male,0.0,AL,2362
4,1994,01,01001,25-34 years,Female,0.0,AL,2938
...,...,...,...,...,...,...,...,...
69415,1994,56,56045,75-84 years,Male,0.0,WY,116
69416,1994,56,56045,85 years and over,Female,0.0,WY,71
69417,1994,56,56045,85 years and over,Male,0.0,WY,28
69418,1994,56,56045,Under 1 year,Female,0.0,WY,36


In [88]:
all_death_pop

,year,statefips,fips,age,deaths,state,pop
0,1994,01,01001,1-4 years,0.0,AL,2350
1,1994,01,01001,15-24 years,0.0,AL,4747
2,1994,01,01001,25-34 years,0.0,AL,5646
3,1994,01,01001,35-44 years,0.0,AL,5065
4,1994,01,01001,45-54 years,3.0,AL,4508
...,...,...,...,...,...,...,...
34856,1994,56,56045,55-64 years,0.0,WY,618
34857,1994,56,56045,65-74 years,2.0,WY,410
34858,1994,56,56045,75-84 years,0.0,WY,300
34859,1994,56,56045,85 years and over,0.0,WY,99


In [89]:
all_death_pop['statefips'].unique()

array(['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48',
       '49', '50', '51', '53', '54', '55', '56'], dtype=object)

In [90]:
standard_pop = standard_pop.rename(columns = {'Age': 'age'})
standard_pop

,age,Number,Weight
0,All ages,1000000,1.000000
1,Under 1 year,13818,0.013818
2,1-4 years,55317,0.055317
3,5-14 years,145565,0.145565
4,15-24 years,138646,0.138646
5,25-34 years,135573,0.135573
6,35-44 years,162613,0.162613
7,45-54 years,134834,0.134834
8,55-64 years,87247,0.087247
9,65-74 years,66037,0.066037


In [91]:
standard_pop = standard_pop.rename(columns = {'Number  ': 'Number', '   Weight': 'Weight'})

In [92]:
def age_adjusted_rate(df):
    data = []
    for county in list(df['fips'].unique()):
        df_county = df[df['fips'] == county]
        df_county['cube_rate'] = (df_county['deaths'] / df_county['pop']) * 100000
        df_county = df_county.merge(standard_pop, on = ['age'], how = 'inner')
        df_county['cube_rate_weight'] = df_county['cube_rate'] * df_county['Weight']
        age_adjusted_rate = round(sum(df_county['cube_rate_weight']),2)
        df_county['age_adjusted_rate'] = age_adjusted_rate
        data.append(df_county)
        
    df_return = pd.concat(data)
    return df_return

In [93]:
import warnings
warnings.filterwarnings('ignore')

In [94]:
all_death_adjusted = age_adjusted_rate(all_death_pop)
all_death_adjusted

,year,statefips,fips,age,deaths,state,pop,cube_rate,Number,Weight,cube_rate_weight,age_adjusted_rate
0,1994,01,01001,1-4 years,0.0,AL,2350,0.000000,55317,0.055317,0.000000,53.82
1,1994,01,01001,15-24 years,0.0,AL,4747,0.000000,138646,0.138646,0.000000,53.82
2,1994,01,01001,25-34 years,0.0,AL,5646,0.000000,135573,0.135573,0.000000,53.82
3,1994,01,01001,35-44 years,0.0,AL,5065,0.000000,162613,0.162613,0.000000,53.82
4,1994,01,01001,45-54 years,3.0,AL,4508,66.548358,134834,0.134834,8.972981,53.82
...,...,...,...,...,...,...,...,...,...,...,...,...
6,1994,56,56045,55-64 years,0.0,WY,618,0.000000,87247,0.087247,0.000000,32.21
7,1994,56,56045,65-74 years,2.0,WY,410,487.804878,66037,0.066037,32.213171,32.21
8,1994,56,56045,75-84 years,0.0,WY,300,0.000000,44842,0.044842,0.000000,32.21
9,1994,56,56045,85 years and over,0.0,WY,99,0.000000,15508,0.015508,0.000000,32.21


In [95]:
all_death_adjusted_part1 = all_death_adjusted[['year', 'statefips', 'fips', 'state', 'deaths', 'pop']]
all_death_adjusted_part1 = all_death_adjusted_part1.groupby(['year', 'statefips', 'fips', 'state']).sum()
all_death_adjusted_part1 = all_death_adjusted_part1.reset_index()
all_death_adjusted_part1

,year,statefips,fips,state,deaths,pop
0,1994,01,01001,AL,17.0,36098
1,1994,01,01003,AL,87.0,103992
2,1994,01,01005,AL,7.0,24439
3,1994,01,01007,AL,1.0,15785
4,1994,01,01009,AL,21.0,37957
...,...,...,...,...,...,...
3130,1994,56,56037,WY,15.0,37710
3131,1994,56,56039,WY,5.0,13366
3132,1994,56,56041,WY,4.0,18838
3133,1994,56,56043,WY,6.0,7764


In [96]:
all_death_adjusted_part2 = all_death_adjusted[['year', 'statefips', 'fips', 'state', 'age_adjusted_rate']]
all_death_adjusted_part2 = all_death_adjusted_part2.drop_duplicates()
all_death_adjusted_part2

,year,statefips,fips,state,age_adjusted_rate
0,1994,01,01001,AL,53.82
0,1994,01,01003,AL,86.29
0,1994,01,01005,AL,27.65
0,1994,01,01007,AL,6.72
0,1994,01,01009,AL,54.59
...,...,...,...,...,...
0,1994,56,56037,WY,61.69
0,1994,56,56039,WY,71.26
0,1994,56,56041,WY,39.75
0,1994,56,56043,WY,66.66


In [97]:
all_death_adjusted = all_death_adjusted_part1.merge(all_death_adjusted_part2, on = ['year', 'statefips', 'fips', 'state'], how = 'inner')
all_death_adjusted

,year,statefips,fips,state,deaths,pop,age_adjusted_rate
0,1994,01,01001,AL,17.0,36098,53.82
1,1994,01,01003,AL,87.0,103992,86.29
2,1994,01,01005,AL,7.0,24439,27.65
3,1994,01,01007,AL,1.0,15785,6.72
4,1994,01,01009,AL,21.0,37957,54.59
...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,15.0,37710,61.69
3131,1994,56,56039,WY,5.0,13366,71.26
3132,1994,56,56041,WY,4.0,18838,39.75
3133,1994,56,56043,WY,6.0,7764,66.66


In [98]:
all_death_adjusted['cube_rate'] = (all_death_adjusted['deaths']/all_death_adjusted['pop'])*100000
all_death_adjusted

,year,statefips,fips,state,deaths,pop,age_adjusted_rate,cube_rate
0,1994,01,01001,AL,17.0,36098,53.82,47.094022
1,1994,01,01003,AL,87.0,103992,86.29,83.660282
2,1994,01,01005,AL,7.0,24439,27.65,28.642743
3,1994,01,01007,AL,1.0,15785,6.72,6.335128
4,1994,01,01009,AL,21.0,37957,54.59,55.325763
...,...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,15.0,37710,61.69,39.777247
3131,1994,56,56039,WY,5.0,13366,71.26,37.408350
3132,1994,56,56041,WY,4.0,18838,39.75,21.233677
3133,1994,56,56043,WY,6.0,7764,66.66,77.279753


### Calculate female age adjusted rates

In [99]:
gender_death_pop

,year,statefips,fips,age,sex,deaths,state,pop
0,1994,01,01001,1-4 years,Female,0.0,AL,1157
1,1994,01,01001,1-4 years,Male,0.0,AL,1193
2,1994,01,01001,15-24 years,Female,0.0,AL,2385
3,1994,01,01001,15-24 years,Male,0.0,AL,2362
4,1994,01,01001,25-34 years,Female,0.0,AL,2938
...,...,...,...,...,...,...,...,...
69415,1994,56,56045,75-84 years,Male,0.0,WY,116
69416,1994,56,56045,85 years and over,Female,0.0,WY,71
69417,1994,56,56045,85 years and over,Male,0.0,WY,28
69418,1994,56,56045,Under 1 year,Female,0.0,WY,36


In [100]:
female_death_pop = gender_death_pop[gender_death_pop['sex'] == 'Female']
male_death_pop = gender_death_pop[gender_death_pop['sex'] == 'Male']

In [101]:
female_death_adjusted = age_adjusted_rate(female_death_pop)
female_death_adjusted

,year,statefips,fips,age,sex,deaths,state,pop,cube_rate,Number,Weight,cube_rate_weight,age_adjusted_rate
0,1994,01,01001,1-4 years,Female,0.0,AL,1157,0.000000,55317,0.055317,0.000000,46.57
1,1994,01,01001,15-24 years,Female,0.0,AL,2385,0.000000,138646,0.138646,0.000000,46.57
2,1994,01,01001,25-34 years,Female,0.0,AL,2938,0.000000,135573,0.135573,0.000000,46.57
3,1994,01,01001,35-44 years,Female,0.0,AL,3245,0.000000,162613,0.162613,0.000000,46.57
4,1994,01,01001,45-54 years,Female,2.0,AL,2386,83.822297,134834,0.134834,11.302096,46.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,1994,56,56045,55-64 years,Female,0.0,WY,317,0.000000,87247,0.087247,0.000000,0.00
7,1994,56,56045,65-74 years,Female,0.0,WY,143,0.000000,66037,0.066037,0.000000,0.00
8,1994,56,56045,75-84 years,Female,0.0,WY,184,0.000000,44842,0.044842,0.000000,0.00
9,1994,56,56045,85 years and over,Female,0.0,WY,71,0.000000,15508,0.015508,0.000000,0.00


In [102]:
def formatAgeAdjustedData(df, df_type):
    df_part1 = df[['year', 'statefips', 'fips', 'state', df_type, 'deaths', 'pop']]
    df_part1 = df_part1.groupby(['year', 'statefips', 'fips', 'state', df_type]).sum()
    df_part1 = df_part1.reset_index()
    df_part2 = df[['year', 'statefips', 'fips', 'state', df_type, 'age_adjusted_rate']]
    df_part2 = df_part2.drop_duplicates()
    df_new = df_part1.merge(df_part2, on = ['year', 'statefips', 'fips', 'state', df_type], how = 'inner')
    return df_new

In [103]:
all_death_adjusted

,year,statefips,fips,state,deaths,pop,age_adjusted_rate,cube_rate
0,1994,01,01001,AL,17.0,36098,53.82,47.094022
1,1994,01,01003,AL,87.0,103992,86.29,83.660282
2,1994,01,01005,AL,7.0,24439,27.65,28.642743
3,1994,01,01007,AL,1.0,15785,6.72,6.335128
4,1994,01,01009,AL,21.0,37957,54.59,55.325763
...,...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,15.0,37710,61.69,39.777247
3131,1994,56,56039,WY,5.0,13366,71.26,37.408350
3132,1994,56,56041,WY,4.0,18838,39.75,21.233677
3133,1994,56,56043,WY,6.0,7764,66.66,77.279753


In [104]:
female_age_adjusted = formatAgeAdjustedData(female_death_adjusted, 'sex')
female_age_adjusted

,year,statefips,fips,state,sex,deaths,pop,age_adjusted_rate
0,1994,01,01001,AL,Female,8.0,19251,46.57
1,1994,01,01003,AL,Female,42.0,50240,96.17
2,1994,01,01005,AL,Female,4.0,12104,31.78
3,1994,01,01007,AL,Female,0.0,8401,0.00
4,1994,01,01009,AL,Female,4.0,17158,20.72
...,...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,Female,8.0,19055,63.00
3131,1994,56,56039,WY,Female,2.0,6281,51.95
3132,1994,56,56041,WY,Female,3.0,8933,51.50
3133,1994,56,56043,WY,Female,0.0,3805,0.00


In [105]:
male_death_adjusted = age_adjusted_rate(male_death_pop)
male_age_adjusted = formatAgeAdjustedData(male_death_adjusted, 'sex')
male_age_adjusted

,year,statefips,fips,state,sex,deaths,pop,age_adjusted_rate
0,1994,01,01001,AL,Male,9.0,16847,58.63
1,1994,01,01003,AL,Male,45.0,53752,80.31
2,1994,01,01005,AL,Male,3.0,12335,23.78
3,1994,01,01007,AL,Male,1.0,7384,22.16
4,1994,01,01009,AL,Male,17.0,20799,86.52
...,...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,Male,7.0,18655,60.11
3131,1994,56,56039,WY,Male,3.0,7085,83.37
3132,1994,56,56041,WY,Male,1.0,9905,15.17
3133,1994,56,56043,WY,Male,6.0,3959,137.45


In [106]:
white_death_pop = race_death_pop[race_death_pop['race'] == 'White']
black_death_pop = race_death_pop[race_death_pop['race'] == 'Black']
other_death_pop = race_death_pop[race_death_pop['race'] == 'Other']

In [107]:
white_death_adjusted = age_adjusted_rate(white_death_pop)
white_age_adjusted = formatAgeAdjustedData(white_death_adjusted, 'race')
white_age_adjusted

,year,statefips,fips,state,race,deaths,pop,age_adjusted_rate
0,1994,01,01001,AL,White,16.0,29522,61.22
1,1994,01,01003,AL,White,80.0,90058,87.07
2,1994,01,01005,AL,White,7.0,13423,41.31
3,1994,01,01007,AL,White,0.0,12402,0.00
4,1994,01,01009,AL,White,21.0,37324,55.32
...,...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,White,15.0,36845,63.01
3131,1994,56,56039,WY,White,5.0,13203,71.37
3132,1994,56,56041,WY,White,4.0,18622,39.86
3133,1994,56,56043,WY,White,6.0,7692,67.17


In [108]:
black_death_adjusted = age_adjusted_rate(black_death_pop)
black_age_adjusted = formatAgeAdjustedData(black_death_adjusted, 'race')
black_age_adjusted

,year,statefips,fips,state,race,deaths,pop,age_adjusted_rate
0,1994,01,01001,AL,Black,1.0,6330,28.21
1,1994,01,01003,AL,Black,7.0,12923,77.88
2,1994,01,01005,AL,Black,0.0,10911,0.00
3,1994,01,01007,AL,Black,1.0,3367,34.76
4,1994,01,01009,AL,Black,0.0,453,0.00
...,...,...,...,...,...,...,...,...
2876,1994,56,56035,WY,Black,0.0,2,0.00
2877,1994,56,56037,WY,Black,0.0,293,0.00
2878,1994,56,56039,WY,Black,0.0,20,0.00
2879,1994,56,56041,WY,Black,0.0,24,0.00


In [109]:
other_death_adjusted = age_adjusted_rate(other_death_pop)
other_age_adjusted = formatAgeAdjustedData(other_death_adjusted, 'race')
other_age_adjusted

,year,statefips,fips,state,race,deaths,pop,age_adjusted_rate
0,1994,01,01001,AL,Other,0.0,246,0.0
1,1994,01,01003,AL,Other,0.0,1011,0.0
2,1994,01,01005,AL,Other,0.0,105,0.0
3,1994,01,01007,AL,Other,0.0,16,0.0
4,1994,01,01009,AL,Other,0.0,180,0.0
...,...,...,...,...,...,...,...,...
3059,1994,56,56037,WY,Other,0.0,572,0.0
3060,1994,56,56039,WY,Other,0.0,143,0.0
3061,1994,56,56041,WY,Other,0.0,192,0.0
3062,1994,56,56043,WY,Other,0.0,68,0.0


In [110]:
def renameData(df, column_name):
    df_new = df.rename(columns = {'deaths': column_name + '_deaths', 'pop': column_name + '_pop', 'age_adjusted_rate': column_name + '_age_adjusted_rate'})
    return df_new

In [111]:
female_age_adjusted_new = renameData(female_age_adjusted, 'female')
male_age_adjusted_new = renameData(male_age_adjusted, 'male')
white_age_adjusted_new = renameData(white_age_adjusted, 'white')
black_age_adjusted_new = renameData(black_age_adjusted, 'black')
other_age_adjusted_new = renameData(other_age_adjusted, 'other_race')

In [112]:
female_age_adjusted_new = female_age_adjusted_new.drop('sex', axis =1)
male_age_adjusted_new = male_age_adjusted_new.drop('sex', axis =1)
white_age_adjusted_new = white_age_adjusted_new.drop('race', axis = 1)
black_age_adjusted_new = black_age_adjusted_new.drop('race', axis = 1)
other_age_adjusted_new = other_age_adjusted_new.drop('race', axis = 1)

In [113]:
all_death_adjusted_new = renameData(all_death_adjusted, 'total') 
all_death_adjusted_new

,year,statefips,fips,state,total_deaths,total_pop,total_age_adjusted_rate,cube_rate
0,1994,01,01001,AL,17.0,36098,53.82,47.094022
1,1994,01,01003,AL,87.0,103992,86.29,83.660282
2,1994,01,01005,AL,7.0,24439,27.65,28.642743
3,1994,01,01007,AL,1.0,15785,6.72,6.335128
4,1994,01,01009,AL,21.0,37957,54.59,55.325763
...,...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,15.0,37710,61.69,39.777247
3131,1994,56,56039,WY,5.0,13366,71.26,37.408350
3132,1994,56,56041,WY,4.0,18838,39.75,21.233677
3133,1994,56,56043,WY,6.0,7764,66.66,77.279753


In [114]:
all_death_adjusted_new = all_death_adjusted_new.drop('cube_rate', axis = 1)

In [115]:
df_final = all_death_adjusted_new.merge(female_age_adjusted_new, on = ['year', 'statefips', 'fips', 'state'], how = 'left')
df_final = df_final.merge(male_age_adjusted_new, on = ['year', 'statefips', 'fips', 'state'], how = 'left')
df_final = df_final.merge(white_age_adjusted_new, on = ['year', 'statefips', 'fips', 'state'], how = 'left')
df_final = df_final.merge(black_age_adjusted_new, on = ['year', 'statefips', 'fips', 'state'], how = 'left')
df_final = df_final.merge(other_age_adjusted_new, on = ['year', 'statefips', 'fips', 'state'], how = 'left')
df_final

,year,statefips,fips,state,total_deaths,total_pop,total_age_adjusted_rate,female_deaths,female_pop,female_age_adjusted_rate,...,male_age_adjusted_rate,white_deaths,white_pop,white_age_adjusted_rate,black_deaths,black_pop,black_age_adjusted_rate,other_race_deaths,other_race_pop,other_race_age_adjusted_rate
0,1994,01,01001,AL,17.0,36098,53.82,8.0,19251,46.57,...,58.63,16.0,29522,61.22,1.0,6330.0,28.21,0.0,246.0,0.0
1,1994,01,01003,AL,87.0,103992,86.29,42.0,50240,96.17,...,80.31,80.0,90058,87.07,7.0,12923.0,77.88,0.0,1011.0,0.0
2,1994,01,01005,AL,7.0,24439,27.65,4.0,12104,31.78,...,23.78,7.0,13423,41.31,0.0,10911.0,0.00,0.0,105.0,0.0
3,1994,01,01007,AL,1.0,15785,6.72,0.0,8401,0.00,...,22.16,0.0,12402,0.00,1.0,3367.0,34.76,0.0,16.0,0.0
4,1994,01,01009,AL,21.0,37957,54.59,4.0,17158,20.72,...,86.52,21.0,37324,55.32,0.0,453.0,0.00,0.0,180.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3130,1994,56,56037,WY,15.0,37710,61.69,8.0,19055,63.00,...,60.11,15.0,36845,63.01,0.0,293.0,0.00,0.0,572.0,0.0
3131,1994,56,56039,WY,5.0,13366,71.26,2.0,6281,51.95,...,83.37,5.0,13203,71.37,0.0,20.0,0.00,0.0,143.0,0.0
3132,1994,56,56041,WY,4.0,18838,39.75,3.0,8933,51.50,...,15.17,4.0,18622,39.86,0.0,24.0,0.00,0.0,192.0,0.0
3133,1994,56,56043,WY,6.0,7764,66.66,0.0,3805,0.00,...,137.45,6.0,7692,67.17,0.0,4.0,0.00,0.0,68.0,0.0


In [116]:
df_final.to_csv('county_level_age_adjusted_rates/age_adjusted_rates_' + str(year) + '.csv')